In [ ]:
import pathlib

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

# Download and extract the dataset
dataset = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
directory = tf.keras.utils.get_file('flower_photos', origin=dataset, untar=True)
data_directory = pathlib.Path(directory)

In [22]:
# Define the image size and batch size
img_height, img_width = 180, 180
batch_size = 32

# Create the training and validation datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_directory,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_directory,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Get the class names
classnames = train_ds.class_names

# Create the ResNet50 model and set the layers to be non-trainable
resnet_model = tf.keras.Sequential()
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(img_height, img_width, 3),
    pooling='avg',
    weights='imagenet'
)

for layer in pretrained_model.layers:
    layer.trainable = False
resnet_model.add(pretrained_model)

# Add fully connected layers for classification
resnet_model.add(layers.Flatten())
resnet_model.add(layers.Dense(512, activation='relu'))
resnet_model.add(layers.Dense(len(classnames), activation='softmax'))

# Compile and train the model using sparse_categorical_crossentropy
resnet_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = resnet_model.fit(train_ds, validation_data=validation_ds, epochs=2)


In [23]:

import cv2
import numpy as np

# Model Inference
# Load and preprocess the sample image

frame_path = ['data/rose.jpg', 'data/sunflower.jpg', 'data/tulip.jpg',
              'data/dandelion.jpg']  # Replace with the path to your image
for frame_path in frame_path:
    image = cv2.imread(frame_path)
    image_resized = cv2.resize(image, (img_height, img_width))  # Resize the image
    image = np.expand_dims(image_resized, axis=0)  # Expand dimensions for batch size

    # Normalize the image to match the training data
    image = image / 255.0  # Rescale the image to [0, 1] if needed

    # Make predictions
    image_pred = resnet_model.predict(image)

    # Produce a human-readable output label
    image_output_class = classnames[np.argmax(image_pred)]
    print("The predicted class is", image_output_class)


In [24]:
# Save the model in TensorFlow's SavedModel format
resnet_model.save('./resnet_model/resnet_model.keras')

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
import cv2
import numpy as np

# Define your image processing parameters
img_height, img_width = 224, 224  # Ensure these match the model's expected input dimensions

# List of image paths
frame_paths = ['data/rose.jpg', 'data/sunflower.jpg', 'data/tulip.jpg', 'data/dandelion.jpg']

# Load the original model
model = tf.keras.models.load_model('./resnet_model.keras')

# Create a list to hold the submodels
submodels = []

# Define the input tensor
input_tensor = Input(shape=(img_height, img_width, 3))

# Iterate through layers and create submodels
x = input_tensor  # Start with the input tensor
for layer in model.layers:
    # Pass the input through the current layer to build the submodel
    x = layer(x)
    submodel = Model(inputs=input_tensor, outputs=x)
    submodels.append(submodel)

# Loop over the images and process each one through the submodels
for frame_path in frame_paths:
    # Read and preprocess the image
    image = cv2.imread(frame_path)
    if image is None:
        print(f"Failed to load image: {frame_path}")
        continue

    # Resize the image to the correct dimensions
    image_resized = cv2.resize(image, (img_width, img_height))
    image_resized = np.expand_dims(image_resized, axis=0)  # Expand dimensions for batch size

    # Normalize the image
    image_resized = image_resized / 255.0  # Rescale the image to [0, 1]

    # Pass the image through each submodel
    for i, submodel in enumerate(submodels):
        # Get the output from the current submodel
        layer_output = submodel.predict(image_resized)
        
        # Handle outputs of different shapes
        if len(layer_output.shape) == 4:  # If the output is an image-like tensor
            output_image = layer_output[0, :, :, :]
            # Normalize the output for visualization
            output_image = (output_image - np.min(output_image)) / (np.max(output_image) - np.min(output_image))
        elif len(layer_output.shape) == 2:  # Dense layers output 2D tensors
            print(f"Output of submodel {i} ({submodel.layers[-1].name}): {layer_output}")
        
        # Update the input for the next submodel
        image_resized = layer_output



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Output of submodel 0 (resnet50): [[0.         0.         0.00494438 ... 2.1174688  0.         0.        ]]


ValueError: Input 0 of layer "functional_37" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(1, 2048)